<a href="https://colab.research.google.com/github/Alessa2005/Practicas-Programacion/blob/main/Proyecto_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

class Grafica():
    """Representa una gráfica con una matriz de adyacencia."""

    def __init__(self, nodos, arcos):
        self.nodos = nodos
        self.matriz = np.zeros((len(nodos), len(nodos)))
        for x in arcos:
            self.matriz[x[0], x[1]] = 1

    def __str__(self):
        return str(self.matriz)

    def agregar_arco(self, tupla):
        """Agrega un arco de `tupla[0]` a `tupla[1]`."""
        self.matriz[tupla[0], tupla[1]] = 1

    def quitar_arco(self, tupla):
        """Quita un arco de `tupla[0]` a `tupla[1]`."""
        self.matriz[tupla[0], tupla[1]] = 0

    def agregar_nodo(self):
        """Agrega un nuevo nodo a la gráfica."""
        vertical = np.zeros((len(self.matriz), 1))
        horizontal = np.zeros((1, len(self.matriz) + 1))
        self.matriz = np.vstack(
            (np.hstack((self.matriz, vertical)), horizontal))

    def quitar_nodo(self, nodo):
        """Quita un nodo de la gráfica y sus conexiones."""
        self.matriz = np.delete(self.matriz, nodo, axis=0)
        self.matriz = np.delete(self.matriz, nodo, axis=1)

    def num_nodos(self):
        """Retorna el número de nodos en la gráfica."""
        return len(self.matriz)

    def num_arcos(self):
        """Retorna el número de arcos en la gráfica."""
        return int(self.matriz.sum())

In [ ]:
class Red(Grafica):
    """Representa una red que hereda de la clase Grafica y añade el cálculo
    de la matriz de transición."""

    def __init__(self, nodos, arcos):
        super().__init__(nodos, arcos)
        self.matriz_transicion = self.calcular_matriz_transicion()

    def calcular_matriz_transicion(self):
        """Calcula la matriz de transición normalizada."""
        matriz_transicion = np.zeros_like(self.matriz)
        for i in range(len(self.matriz)):
            if self.matriz[i, :].sum() > 0:
                matriz_transicion[i, :] = (
                    self.matriz[i, :] / self.matriz[i, :].sum())
        return matriz_transicion


In [ ]:
class PageRank():
    """Implementa el algoritmo PageRank para calcular la importancia de las
    páginas en una red."""

    def __init__(self, red, paginas, d=0.85, tol=1e-6):
        self.red = red
        self.paginas = paginas
        self.d = d
        self.tol = tol
        self.pi = np.ones((1, len(self.red.matriz))) / len(self.red.matriz)

    def es_ru(self, pagina):
        """Verifica si la página termina en '.ru'."""
        return pagina.endswith('.ru')

    def inicializar_pi(self, tipo="uniforme"):
        """Inicializa el vector pi según el tipo especificado (uniforme o
        condicionado por .ru)."""
        if tipo == "ru":
            s = sum(1 for p in self.paginas if self.es_ru(p))
            for k in range(len(self.paginas)):
                if self.es_ru(self.paginas[k]):
                    self.pi[0, k] = 1 / s
                else:
                    self.pi[0, k] = 0
        elif tipo == "uniforme":
            self.pi = np.ones((1, len(self.red.matriz))) / len(self.red.matriz)

    def ejecutar(self):
        """Ejecuta el algoritmo iterativo de PageRank hasta la convergencia."""
        pi_1 = self.pi @ self.red.matriz_transicion
        while np.linalg.norm(self.pi - pi_1) > self.tol:
            self.pi = pi_1
            pi_1 = self.pi @ self.red.matriz_transicion
        return self.pi

    def calcular_u(self):
        """Calcula la matriz U con entradas 1/n (distribución uniforme)."""
        n = len(self.red.matriz)
        return np.ones((n, n)) / n

    def ejecutar_iterativo(self):
        """Ejecuta la iteración modificada para PageRank con la fórmula dada."""
        u = self.calcular_u()
        pi_1 = self.pi
        while True:
            pi_2 = pi_1 @ (self.d * self.red.matriz_transicion + (1 - self.d) * u)
            if np.linalg.norm(pi_2 - pi_1) < self.tol:
                break
            pi_1 = pi_2
        return pi_2

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount("mnt")

def construccion_pd(ruta_nombre_del_archivo):
    """Construye una red a partir de un archivo Excel con la información de las
    páginas y sus enlaces."""
    df = pd.read_excel(ruta_nombre_del_archivo)
    nodos = list(range(len(df["Index"])))
    arcos = []
    for i, enlaces in enumerate(df["Cited by"]):
        for j in enlaces.split(','):
            arcos.append((int(j) - 1, i))
    paginas = list(df["Website"])
    return Red(nodos, arcos), paginas

Mounted at mnt


In [ ]:
def respuestas():
    ruta_del_archivo_excel = (
        "/content/mnt/MyDrive/Classroom/Programación 9292/Copia de Web.xlsx"
    )

    # Paso 1: Construir la red
    red, paginas = construccion_pd(ruta_del_archivo_excel)

    # Paso 2: Inicializar PageRank con el vector uniforme
    page_rank = PageRank(red, paginas)
    page_rank.inicializar_pi(tipo="uniforme")

    # Ejecutar PageRank con el vector uniforme
    resultado = page_rank.ejecutar()

    # Página más importante con el vector uniforme
    indice_max = np.argmax(resultado)
    pagina_importante = paginas[indice_max]
    print(f"Página más importante con vector uniforme: {pagina_importante}")

    # Paso 3: Inicializar el vector pi con la condición de páginas .ru
    page_rank.inicializar_pi(tipo="ru")
    resultado_ru = page_rank.ejecutar()

    # Página más importante con el vector pi basado en .ru
    indice_max_ru = np.argmax(resultado_ru)
    pagina_importante_ru = paginas[indice_max_ru]
    print(f"Página más importante con vector pi basado en .ru: {pagina_importante_ru}")


if __name__ == "__main__":
    respuestas()


Página más importante con vector uniforme: federalreserve.gov
Página más importante con vector pi basado en .ru: vedomosti.ru
